# Наполнение базы данных.

Данные для наполнения берутся с сайта [викиконспектов](https://neerc.ifmo.ru/wiki/index.php?title=%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0) от университета ИТМО. Скачиваемые данные очищаются от HTML-тегов, пишутся в текстовые файлы по темам, которые кладутся в директорию `text_db`.

In [15]:
%pip install requests beautifulsoup4 mwparserfromhell

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/100.5 kB ? eta -:--:--
     ----------- ------------------------- 30.7/100.5 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 100.5/100.5 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\savch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
from bs4 import BeautifulSoup
import mwparserfromhell

def get_text(node,trim=False):
    if node is None:
        return ""
    res = node.find_all("div", recursive=False)
    if res:
        if trim:
            res = list(res)[1:-1]
        return '\n'.join(get_text(part) for part in res)
    else:
        return node.get_text()
    
def extract(html):
    soup = BeautifulSoup(html, "html.parser")
    art = soup.find("article")
    return get_text(html,True)

def clean_wikitext(wikitext):
    wikicode = mwparserfromhell.parse(wikitext)
    text = wikicode.strip_code()
    return text

In [17]:
hash_titles = [
    "Хеш-таблица",
    "Разрешение_коллизий",
    "Хеширование_кукушки",
    "Идеальное_хеширование",
    "Перехеширование",
    "Фильтр_Блума",
    "Универсальное_семейство_хеш-функций"
]

In [22]:
import requests

url = "https://neerc.itmo.ru/wiki/api.php"

def get_web_content(title):
    parameters = {
       "action": "query",
        "prop": "revisions",
        "titles": title,
        "rvprop": "content",
        "format": "json"
    }

    response = requests.get(url, params=parameters)
    data = response.json()
    pages = data['query']['pages']
    for page_id, page_info in pages.items():
        if page_id != "-1": 
            return page_info['revisions'][0]['*']
        else:
            return None

In [23]:
def write_to_file(conspects, filename):
    with open(filename, "w", encoding="utf-8") as conspects_file:
        for title in conspects:
            conspects_file.write(conspects[title])
            conspects_file.write('-----')

In [24]:
def get_content(titles):
    content = {
        title: "" for title in titles
    }
    for title in titles:
        content[title] = clean_wikitext(get_web_content(title))
        print(content[title])

    return content

In [25]:
content = get_content(hash_titles)
write_to_file(content, "text_db/hashing.txt")


 Введение 
Существует два основных вида хеш-таблиц: с цепочками и открытой адресацией. Хеш-таблица содержит некоторый массив H, элементы которого есть пары (хеш-таблица с открытой адресацией) или списки пар (хеш-таблица с цепочками).

Выполнение операции в хеш-таблице начинается с вычисления хеш-функции от ключа. Хеш-код i = h(key) играет роль индекса в массиве H, а зная индекс, мы можем выполнить требующуюся операцию (добавление, удаление или поиск).

Количество коллизий зависит от хеш-функции; чем лучше используемая хеш-функция, тем меньше вероятность их возникновения. 

Способ разрешения коллизий — важная составляющая любой хеш-таблицы.

Полностью избежать коллизий для произвольных данных невозможно в принципе, и хорошая хеш-функция в состоянии только минимизировать их количество. Но, в некоторых специальных случаях их удаётся избежать. Если все ключи элементов известны заранее, либо меняются очень редко, то можно подобрать хеш-функцию, с помощью которой, все ключи будут распределен

In [26]:

sorting_titles = [
    "Сортировка_выбором",
    "Сортировка_пузырьком",
    "Сортировка_вставками",
    "Сортировка_кучей",
    "Сортировка_Шелла",
    "Быстрая_сортировка",
    "Сортировка_слиянием",
    "Теорема_о_нижней_оценке_для_сортировки_сравнениями",
    "Сортировка_подсчётом",
    "Цифровая_сортировка",
    "Поиск_k-ой_порядковой_статистики"
]

In [27]:
content = get_content(sorting_titles)
write_to_file(content, "text_db/sorting.txt")

Сортировка выбором (англ. selection sort)  простой алгоритм сортировки со сложностью O(n^2), где n  количество элементов для сортировки.

 Алгоритм 
На каждом i-ом шаге алгоритма находим i-ый минимальный элемент и меняем его местами с i-ым элементом в массиве. Таким образом будет получен массив, отсортированный по неубыванию.

 Псевдокод 
Вариант 1.
Будем каждый раз проходить по всем еще не отсортированным элементам, и, как только найдем элемент меньше, чем первый из неотсортированных, поменяем их местами. Таким образом будет нужно O(n^2) обменов (для каждого i требуется O(n-i) обменов). 
  function selectionSort(T[n] a):
    for i = 0 to n - 2
      for j = i + 1 to n - 1
        if a[i] > a[j]
          swap(a[i], a[j])

Вариант 2.
Второй вариант немного более экономный. Здесь мы будем менять местами элементы только 1 раз для каждого i, всего будет нужно O(n) обменов. Для этого сначала мы будем проходить по всем еще не отсортированным элементам, искать минимальный, и только потом мен